In [1]:
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
from torch_scatter import scatter_max, scatter_mean
import lightning as L
from torch_geometric.data import Batch, Data
from tqdm import tqdm
import re
from torch import nn
from torch.utils.flop_counter import FlopCounterMode
from transformers import AutoModel, AutoTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 64
folder_path = r'C:\Users\fardin\Projects\CGNet\Data\TextClassification\IMDB'
# t_tokenizer = TweetTokenizer()
device

device(type='cuda')

In [2]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-large")
model = AutoModel.from_pretrained("microsoft/deberta-v3-large")
id_vocab = {v:k for k,v in tokenizer.vocab.items()}
all_vocab_indices = list(id_vocab.keys())

with open(r'Data\ReducedEmbeddings\deberta_larg_reduced_embeddings_64.npy', 'rb') as f:
    embeddings = np.load(f)
embeddings = torch.from_numpy(embeddings)
all_vocab_str = []
for i in range(len(id_vocab)):
    all_vocab_str.append(id_vocab[i])
token_vocab_dict = dict(zip(all_vocab_str, embeddings))

with open(r'Data\ReducedEmbeddings\polarity_debertav3_tokens_gpt_mini_emb.npy', 'rb') as f:
    polarities_subjectivities= np.load(f)
polarities_subjectivities = torch.from_numpy(polarities_subjectivities)
polarity_vocab_dict = dict(zip(all_vocab_str, polarities_subjectivities))
polarity_vocab_dict['<n>'] = torch.tensor([0.0, 0.0])
len(token_vocab_dict)
polarities_subjectivities.shape
for i in range(len(all_vocab_str)):
    if 'nice' in all_vocab_str[i]:
        print(i, polarities_subjectivities[i])
        break

c:\Users\fardin\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


1085 tensor([0.7000, 0.6000], dtype=torch.float64)


In [3]:
keep_ratio = 0.1
test_df = pd.read_csv(r'data\TextClassification\IMDB\test.csv')
test_df['Topic'] = test_df['label']
test_df['Content'] = test_df['text']
test_df.drop(['label', 'text'], axis=1, inplace=True)
test_df.dropna(inplace=True)
test_df = test_df.sample(frac=1, random_state=42).reset_index(drop=True)
test_df = test_df.iloc[:int(keep_ratio*test_df.shape[0])]
train_df = pd.read_csv(r'data\TextClassification\IMDB\train.csv')
train_df['Topic'] = train_df['label']
train_df['Content'] = train_df['text']
train_df.drop(['label', 'text'], axis=1, inplace=True)
train_df.dropna(inplace=True)
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
train_df = train_df.iloc[:int(keep_ratio*train_df.shape[0])]
sst_classes = ["Negative", "Positive"]
df = pd.DataFrame(np.concatenate([train_df.values, test_df.values]), columns=train_df.columns)
class_id = {c:i for i, c in enumerate(sst_classes)}
id_class = {i:c for i, c in enumerate(sst_classes)}
df.shape

(5000, 2)

In [4]:
# %%time
vocabs_lists = list(token_vocab_dict.keys())
term_frequencies = {t:1 for t in vocabs_lists}
temp_term_frequencies = {}

for doc in train_df.Content.values:
    tokens_list = tokenizer.tokenize(doc)
    new_tokens = {t.strip('▁').lower() for t in tokens_list}
    for t in new_tokens:
        if t not in temp_term_frequencies:
            temp_term_frequencies[t] = 0
        temp_term_frequencies[t] += 1
        
for k, v in term_frequencies.items():
    stripped_token = k.strip('▁').lower()
    term_frequencies[k] = temp_term_frequencies[stripped_token] if stripped_token in temp_term_frequencies else 1


In [5]:
threshold = 0.00001
total_token_count = np.array(list(term_frequencies.values())).sum()
one_tensor = torch.tensor(1)
def subsampling_equation_linear(x: torch.Tensor):
    f_x = x/total_token_count
    x = torch.min(one_tensor, torch.sqrt_(threshold/f_x))
    return x

def subsampling_equation_sigmoid(x: torch.Tensor):
    f_x = x/total_token_count
    x = 1-0.95*F.sigmoid(0.05*((f_x/threshold)-90))
    return x

In [6]:
from utilities.data_manager.CharacterandTokenLevelCustomDataset import CharacterandTokenLevelCustomDataset
from utilities.data_manager.CharacterandTokenLevelDataLoader import CharacterandTokenLevelDataLoader

In [7]:
%%time
train_dataset = CharacterandTokenLevelCustomDataset(train_df.Content.values, train_df.Topic.values, len(class_id), token_vocab_dict, polarity_vocab_dict, tokenizer.tokenize, token_frequencies=term_frequencies, sampling_equation=subsampling_equation_sigmoid, id_class=id_class, batch_size=batch_size)
test_dataset = CharacterandTokenLevelCustomDataset(test_df.Content.values, test_df.Topic.values, len(class_id), token_vocab_dict, polarity_vocab_dict, tokenizer.tokenize, token_frequencies=term_frequencies, sampling_equation=subsampling_equation_sigmoid, id_class=id_class, batch_size=batch_size)
max_token_count = max(train_dataset.max_token_count, test_dataset.max_token_count)
train_dataloader = CharacterandTokenLevelDataLoader(train_dataset, batch_size=batch_size, drop_last=False, shuffle=True)
test_dataloader = CharacterandTokenLevelDataLoader(test_dataset, batch_size=batch_size, drop_last=False, shuffle=False)

  0%|          | 0/2560 [00:00<?, ?it/s]

100%|██████████| 2560/2560 [00:39<00:00, 65.08it/s]

CPU times: total: 4min 59s
Wall time: 1min 21s


In [8]:
X, y = next(iter(test_dataloader))

In [9]:
from utilities.model_layers.GCNN import GCNN
from utilities.model_layers.GenGraph import GenGraph
from utilities.model_layers.SentimentInjection import SentimentInjection

In [10]:
from torch_geometric.nn import GATv2Conv, SimpleConv, summary
import math

class CNN_for_Text_No_Positional_Encoding(nn.Module):
    
    def __init__(self, embedding_dim=64, hidden_dim=64, dropout=0.3, num_out_features=4, seed=-1, random_edges=4, lattice_edges=10, virtual_nodes=1, lattice_step=2, lattice_start_distance=2, inject_embedding_dim=64, isXaiTests=False, step_of_test = 0, num_tests=50, *args, **kwargs) -> None:
        super(CNN_for_Text_No_Positional_Encoding, self).__init__(*args, **kwargs)
        # self.pos_emb_size = pos_emb_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.virtual_nodes = virtual_nodes
        self.base_random_edges = random_edges
        self.base_lattice_edges = lattice_edges
        self.lattice_start_distance = lattice_start_distance
        self.num_out_features = num_out_features
        self.isXaiTests = int(isXaiTests)
        self.num_tests = num_tests
        self.step_of_test = step_of_test

        if seed>-1:
            torch.manual_seed(seed)
        
        self.embedding = nn.Embedding(16384, embedding_dim)
        
        self.dropout = nn.Dropout(p=dropout)
        
        # self.positional_encoding = nn.Embedding(pos_emb_size, embedding_dim)
        # self.positional_encoding.weight = self.create_positional_encoding()
        
        self.conv1 = nn.Conv1d(embedding_dim, hidden_dim, kernel_size=5, padding=2)
        self.pool1 = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(hidden_dim, hidden_dim, kernel_size=5, padding=2)
        
        # if self.use_token_polarity[0]:
        self.conv3 = nn.Conv1d(2*hidden_dim + 2, hidden_dim, kernel_size=3, padding=1)
        # else:
        # self.conv3 = nn.Conv1d(2*hidden_dim, hidden_dim, kernel_size=3, padding=1)
        
        self.conv4 = nn.Conv1d(hidden_dim, hidden_dim, kernel_size=3, padding=1)
        
        # if self.use_token_polarity[1]:
        self.sentiment1  = SentimentInjection(hidden_dim)
        # if self.use_token_polarity[2]:
        self.sentiment2  = SentimentInjection(hidden_dim)
            
        self.gcnn1 = GCNN(hidden_dim)
        self.gcnn2 = GCNN(hidden_dim+inject_embedding_dim)
        self.graph_generator = GenGraph(hidden_dim, virtual_nodes, lattice_step)
        
        k = 4
        self.fc0 = nn.Linear(hidden_dim , hidden_dim+inject_embedding_dim)
        self.fc1 = nn.Linear(hidden_dim+inject_embedding_dim , hidden_dim * k)
        self.fc2 = nn.Linear(hidden_dim * (2+virtual_nodes) * k , 32)
        self.dropout = nn.Dropout(dropout)
        self.fc_out = nn.Linear(32, self.num_out_features)
        self.max_length = 0
    
    def forward(self, x, edge_index, token_subsampling_probabilities, token_indices, token_sentiments, token_lengths, num_tokens, character_length, token_embeddings, token_positions):
        # cumulative_token_indices = token_indices if not self.isXaiTests else self.caluculate_batch_token_positions(num_tokens, character_length, token_indices)
        cumulative_token_indices = self.caluculate_batch_token_positions(num_tokens, character_length, token_indices)
        
        # print(f'2: {x.shape}')
        x = self.embedding(x)
        # print(f'2.5: {x.shape}')
        x = self.dropout(x)
        # print(f'2.6: {x.shape}')
        x = x.T
        # print(f'2.7: {x.shape}')
        # x = self.refine_shape(1, x, 0)
        x = F.relu(self.conv1(x))
        # print(f'2.8: {x.shape}')
        x = self.refine_shape(1, x, self.hidden_dim, 0)
        # print(f'2.8 refined: {x.shape}')
        x = F.relu(self.conv2(x))
        x = self.refine_shape(2, x, self.hidden_dim, 0)
        # print(f'2.9: {x.shape}')
        x = self.dropout(x)
        # x = self.refine_shape(4, x, 0)
        # print(f'3: {x.shape}')
        x1 = scatter_max(x, cumulative_token_indices, dim=1)[0]
        x2 = scatter_mean(x, cumulative_token_indices, dim=1)

        # if self.use_token_polarity[0]:
        x = torch.cat([x1, x2, token_sentiments.T], dim=0)
        # else:
        # x = torch.cat([x1, x2], dim=0)
            
        # print(f'4: {x.shape}')
        x = F.relu(self.conv3(x))
        # x = x + self.positional_encoding(token_positions).T
        
        # print(f'4: {x.shape}')
        
        # print(f'4.5: {x.shape}, self.hidden_dim: {self.hidden_dim}, self.is_tests_token_level: {self.step_of_test}')
        # if self.isXaiTests and x.shape[0] != self.hidden_dim:
        # x = torch.chunk(x, self.num_tests ** (1 - self.isXaiTests), dim=0)
        x = self.refine_shape(3, x, self.hidden_dim, 0)
        
        # x = torch.chunk(x, (x.shape[0] // self.hidden_dim)**self.is_tests_token_level, dim=0)
        # x = torch.cat(x, dim=1)
        
        # if self.isXaiTests and x.shape[0] != self.hidden_dim:
        #     x = torch.chunk(x, self.num_tests, dim=0)
        #     x = torch.cat(x, dim=1)
        # x = x.reshape(self.hidden_dim, -1)
        # print("abababdadasd")
        # print(f'5: {x.shape}, {edge_index.shape}, {cumulative_token_indices.shape}, {token_sentiments.shape}, {token_lengths.shape}, {num_tokens.shape}, {character_length.shape}, {token_embeddings.shape}')
        # if self.use_token_polarity[1]:
        x = self.sentiment1(x.T, token_sentiments)

        # print(f'6: {x.shape}')
        x = self.refine_shape(4, x.T, self.hidden_dim, 1)
        # print(f'6 refined: {x.shape}')
        rand_edges, lattice_edges = self.base_random_edges, self.base_lattice_edges
            
        graph = self.graph_generator.gen_graph(x, token_subsampling_probabilities, len(token_lengths), num_tokens, rand_edges, lattice_edges, lattice_start_distance=self.lattice_start_distance)
        rand_edges = rand_edges-1
        lattice_edges = lattice_edges-1
        
        # print(f'7: {graph.x.shape}')
        
        doc_token_index = torch.repeat_interleave(torch.arange(len(num_tokens), device=x.device), num_tokens)
        x, edge_weights, edge_index = self.gcnn1(graph.x, graph.edge_index, return_attention_weights = True)
        # print(f'7.1: {x.shape}')
        x = self.refine_shape(5, x, self.hidden_dim, 1)
        # print(f'7.1 2 : {len(torch.cat(edge_weights[1::2], dim=0))}')
        edge_weights = self.refine_edge_weights(edge_weights)
        edge_index = self.refine_edge_index(edge_index)
        
        # edge_weights = edge_weights[1].unsqueeze(-1)
        # print(f'7.5 edge_weights: {edge_weights.shape}, - {graph.edge_index.shape[1]}, {edge_weights.shape[0]}')
        # edge_weights = edge_weights[:min(graph.edge_index.shape[1], edge_weights.shape[0]), 0]
        # edge_weights = edge_weights.squeeze()
        
        edge_weights = edge_weights.unsqueeze(-1)
        # print(f'7.5 edge_weights: {edge_weights.shape}, - {edge_weights.shape[0]}')
        edge_weights = edge_weights[:edge_weights.shape[0], 0]
        
        # print(f'7.6 edge_weights: {edge_weights.shape}')
        # print(f'7.7: {x.shape}')
        x = self.refine_shape(5, x, self.hidden_dim, 1)
        # print(f'7.8 refined: {x.shape}')
        
        graph = self.graph_generator.replace_unimportant_edges(edge_weights, x, edge_index, token_subsampling_probabilities, len(token_lengths), num_tokens, rand_edges, lattice_edges, p_keep=2, lattice_start_distance=self.lattice_start_distance+1)
        
        # print(f'8: {graph.x.shape}')
        
        # if self.use_token_polarity[2]:
        x = self.sentiment2(x, token_sentiments)
          
        # print(f'8.1: {x.shape}')
        x = self.refine_shape(6, x.T, self.hidden_dim, 1)
        # print(f'8.2 refined: {x.shape}')
        
        # print(f'9: {x.shape}')  
        xa = graph.x[:token_embeddings.shape[0]]
        xb = token_embeddings
        x = torch.cat([xa, xb], dim=1)
        # x = torch.cat([graph.x[:g_data.token_embeddings.shape[0]], g_data.token_embeddings], dim=1)
        
        # print(f'10: {x.shape}')  
        x1 = F.relu(self.fc0(graph.x[token_embeddings.shape[0]:]))
        x = torch.cat([x, x1], dim=0)
        
        # print(f'11: {x.shape}')  
        sum1 = torch.sum(edge_weights) + torch.sum(edge_index)
        x, edge_weights, edge_index = self.gcnn2(x, graph.edge_index)
        edge_weights = self.refine_edge_weights(edge_weights)
        sum1 = sum1 + torch.sum(edge_weights) + torch.sum(edge_index) 
        
        # print(f'12: {x.shape}')  
        x = F.elu_(self.fc1(x))
        x1 = scatter_max(x[:len(token_lengths)], doc_token_index, dim=0)[0]
        x2 = scatter_mean(x[:len(token_lengths)], doc_token_index, dim=0)
        vn_embs = x[len(token_lengths):]
        x_for_cat = [x1, x2]
        x_for_cat.extend([vn_embs[i*x1.shape[0]:(i+1)*x1.shape[0]] for i in range(self.virtual_nodes)])
        x = torch.cat(x_for_cat, dim=1)
        
        # print(f'13: {x.shape}')  
        x = F.elu_(self.fc2(x))
        x = self.dropout(x)
        x = self.fc_out(x)
        # print(f'14: {x.shape}')  
        return x + sum1 * 0.0
    
    def caluculate_batch_token_positions(self, num_tokens, character_length, token_indices):
        cumsum_vals = torch.cumsum(num_tokens, dim=0).roll(1)
        cumsum_vals[0] = 0
        additions = torch.repeat_interleave(cumsum_vals, character_length)
        cumulative_token_indices = token_indices + additions
        return cumulative_token_indices
    
    # def create_positional_encoding(self):
    #     position = torch.arange(self.pos_emb_size).unsqueeze(1)
    #     div_term = torch.exp(torch.arange(0, self.hidden_dim, 2) * (-math.log(10000.0) / self.hidden_dim))
    #     pe = torch.zeros(self.pos_emb_size, self.hidden_dim)
    #     pe[:, 0::2] = torch.sin(position * div_term)
    #     pe[:, 1::2] = torch.cos(position * div_term)
    #     return torch.nn.Parameter(pe, requires_grad=False)
    
    def refine_shape(self, test_step, x, num_chunks, section=0):
        x = torch.chunk(x, (x.shape[section] // num_chunks)**(self.step_of_test==test_step), dim=0)
        x = torch.cat(x, dim=1-section)
        return x
        
    def refine_edge_weights(self, edge_weights):
        edge_weights = edge_weights[1::2] + edge_weights[0::2] * 0
        edge_weights = [edge_weights[i] for i in range(len(edge_weights))]
        edge_weights = torch.cat(edge_weights, dim=0)
        return edge_weights
        
    def refine_edge_index(self, edge_index):
        edge_index = torch.cat([edge_index[::2].reshape(1, -1), edge_index[1::2].reshape(1, -1)], dim=0)
        return edge_index

In [11]:
# # for p1 in [False, True]:
# #     for p2 in [False, True]:
# #         for p3 in [False, True]:
# # print(f'\n{p1}, {p2}, {p3}: \n')
# classifier_torch_model = CNN_for_Text_No_Positional_Encoding(hidden_dim=64, embedding_dim=64, dropout=0.2, num_out_features=len(class_id), seed=911, random_edges=4, lattice_edges=4, lattice_step=2, virtual_nodes=0, lattice_start_distance=2, isXaiTests=True, num_tests=len(X.num_tokens)).eval()
# flopt_counter = FlopCounterMode(classifier_torch_model)
# with flopt_counter:
#     classifier_torch_model(X.x, torch.zeros((2, 0)), X.token_subsampling_probabilities, X.token_indices, X.token_sentiments, X.token_lengths, X.num_tokens, X.character_length, X.token_embeddings, X.token_positions)

In [12]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from torchmetrics import ConfusionMatrix

def calculate_metrics(cl_model, dataloader):
    cm = ConfusionMatrix(task="multiclass", num_classes=len(class_id))

    y_pred = []
    y_true = []

    cl_model = cl_model.eval()
    cl_model.to(device)
    for X, y in tqdm(dataloader):
        X = X.to(device)
        with torch.no_grad():
            y_p = cl_model(X)
            y_p = y_p.cpu()
        y_pred.append(y_p)
        y_true.append(y)
    y_pred = torch.cat(y_pred, dim=0)
    y_true = torch.cat(y_true, dim=0)
    y_pred2 = torch.argmax(y_pred, dim=1)
    y_true2 = torch.argmax(y_true, dim=1)
    print(f'classification report: \n {classification_report(y_true2, y_pred2, digits=4)}')
    print(f'confusion matrix:\n {cm(y_pred2, y_true2)}')
    print('================================')


In [13]:
from utilities.managers.ModelManager import ModelManager
from utilities.managers.ClassifierModelManager import ClassifierModelManager
from utilities.lightning_models.CnnGnnClassifierLightningModel import CnnGnnClassifierLightningModel

In [14]:
output_size = 256
hidden_dim = 256
embedding_dim = 256
seed = 911

In [15]:
def train_model(epochs=30, dropout=0.25, weight_decay=0.000012, lr=0.0002, amsgrad=False, fused=True, use_positional_encoder=[False, False, False]):
    torch.autograd.set_detect_anomaly(True)
    classifier_torch_model = CNN_for_Text_No_Positional_Encoding(hidden_dim=hidden_dim, embedding_dim=embedding_dim, dropout=dropout, num_out_features=len(class_id), seed=seed, random_edges=6, lattice_edges=10, lattice_step=2, virtual_nodes=0, lattice_start_distance=2).to(device)
    # optimizer = torch.optim.Adam(classifier_torch_model.parameters(), lr=lr, weight_decay=weight_decay, amsgrad=amsgrad, fused=fused)
    optimizer = torch.optim.AdamW(classifier_torch_model.parameters(), lr=lr, weight_decay=weight_decay, amsgrad=amsgrad, fused=fused)
    # lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50, 100, 150, 200, 250, 300, 350],gamma=0.5, verbose=False)
    lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[15, 20, 30, 40, 45,50,55],gamma=0.5, verbose=False)
    loss_func = torch.nn.BCEWithLogitsLoss()
    classfier_lightning_model = CnnGnnClassifierLightningModel(classifier_torch_model, 
                                                        num_classes=len(class_id),
                                                learning_rate=lr,
                                                batch_size=batch_size,
                                                optimizer=optimizer,
                                                loss_func=loss_func,
                                                lr_scheduler=lr_scheduler,
                                                user_lr_scheduler=True
                                                ).to(device)


    model_manager = ClassifierModelManager(classifier_torch_model, classfier_lightning_model, log_name=f'CNN-GNN_{use_positional_encoder[0]}_{use_positional_encoder[1]}_{use_positional_encoder[2]}',device=device, num_train_epoch=epochs, accumulate_grad_batches=1)
    # trainer = L.Trainer(
    #             # callbacks=callbacks,
    #             max_epochs=epochs,
    #             accelerator= 'gpu' if torch.cuda.is_available() else 'cpu',
    #             logger=CSVLogger(save_dir='logs/', name='log2'), 
    #             num_sanity_val_steps=0,
    #         #     default_root_dir='models\model2_word_embedding-256-2'
    #         )

    train_dataset.reset_params()
    train_dataset.position_j = 0
    test_dataset.reset_params()
    test_dataset.position_j = 0
    
    # train_dataset.section_i = 0
    # train_dataset.each_section_i = np.zeros((train_dataset.num_sections, ), dtype=int)
    # test_dataset.section_i = 0
    # test_dataset.each_section_i = np.zeros((test_dataset.num_sections, ), dtype=int)
    
    model_manager.fit(train_dataloaders=train_dataloader, val_dataloaders=test_dataloader)
    model_manager.save_plot_csv_logger(loss_names=['train_loss_epoch', 'val_loss_epoch'], eval_names=['train_acc_epoch', 'val_acc_epoch'], name_prepend=f'tests_{dropout}_{weight_decay}_{lr}_{amsgrad}_{fused}')
    model_manager.lightning_model.model = model_manager.torch_model.to(device)
    model_manager.save_evaluation(model_manager.lightning_model, test_dataloader, f'{dropout}_{weight_decay}_{lr}]',True, True, True, True, True, True, True, multi_class=True)
    # trainer.fit(classfier_lightning_model, train_dataloaders=train_dataloader, val_dataloaders=test_dataloader)
    classfier_lightning_model.model = classfier_lightning_model.model.eval()
    classfier_lightning_model = classfier_lightning_model.eval()
    calculate_metrics(classfier_lightning_model, test_dataloader)
    model_manager.evaluate_best_models(test_dataloader,True, True, True, True, True, True, True, multi_class=True, model=classifier_torch_model, num_classes=len(class_id))
    return model_manager

In [16]:
torch.cuda.empty_cache()
import gc
gc.collect()

20

In [1]:
model_manager = train_model(70, 0.2, 0.000012, 0.0032, use_positional_encoder=[False, False, False])

In [23]:
from pathlib import Path
p = Path(r'logs/CNN-GNN_False_False_False\version_0\checkpoints\epoch=0-step=4.ckpt.pth')
not p.parent.mkdir(parents=True, exist_ok=True)

True

In [24]:
from sklearn.metrics import classification_report, f1_score, accuracy_score, precision_score, recall_score, confusion_matrix, hinge_loss

def calculatge_metrics(chpt_path, target_data_loader):
        classifier_torch_model = CNN_for_Text_No_Positional_Encoding(hidden_dim=hidden_dim, embedding_dim=embedding_dim, dropout=0.2, num_out_features=len(class_id), seed=seed, random_edges=4, lattice_edges=8, lattice_step=2, virtual_nodes=0, lattice_start_distance=2)
        classifier_torch_model.load_state_dict(torch.load(chpt_path, weights_only=True, map_location="cuda:0"))
        classfier_lightning_model = CnnGnnClassifierLightningModel(classifier_torch_model, 
                                                        num_classes=len(class_id),
                                                batch_size=batch_size,
                                                user_lr_scheduler=True
                                                ).to(device).eval()
        
        mean_infer_acc = []
        mean_infer_f1 = []
        mean_infer_prec = []
        mean_infer_rec = []
        for i in range(5):
            all_ys = []
            all_y_preds = []
            for X, y in target_data_loader:
                with torch.no_grad():
                    y_pred = classfier_lightning_model(X.to(device))
                all_ys.append(torch.argmax(y,dim=1))
                all_y_preds.append(torch.argmax(y_pred.cpu(), dim=1))
            all_ys = torch.concat(all_ys)
            all_y_preds = torch.concat(all_y_preds)
            
            cm = confusion_matrix(all_ys, all_y_preds, labels=list(id_class.keys()))
            
            accuracy = np.sum(np.diag(cm))/ np.sum(cm)
            precision = np.mean(np.diag(cm) / (np.sum(cm, axis=0)+0.000001))
            recall = np.mean(np.diag(cm) / (np.sum(cm, axis=1)+0.000001))
            f1_score = (2*precision*recall)/(precision + recall+0.000001)
            
            mean_infer_acc.append(accuracy)
            mean_infer_f1.append(f1_score)
            mean_infer_prec.append(precision)
            mean_infer_rec.append(recall)
        mean_infer_acc = torch.mean(torch.tensor(mean_infer_acc))
        mean_infer_f1 = torch.mean(torch.tensor(mean_infer_f1))
        mean_infer_prec = torch.mean(torch.tensor(mean_infer_prec))
        mean_infer_rec = torch.mean(torch.tensor(mean_infer_rec))
        return mean_infer_acc, mean_infer_f1, mean_infer_prec, mean_infer_rec, classfier_lightning_model

In [25]:
from os import listdir
from os.path import isfile, join
def get_best_chpt(metrics_path, epoch_numbers):
    epoch_data = pd.read_csv(metrics_path)
    if 'val_acc_epoch' in epoch_data.columns and epoch_data['val_acc_epoch'].notna().any():
        best_chpt = epoch_data.loc[epoch_data['val_acc_epoch'].idxmax()]
    elif 'val_loss_epoch' in epoch_data.columns and epoch_data['val_loss_epoch'].notna().any():
        best_chpt = epoch_data.loc[epoch_data['val_loss_epoch'].idxmin()]
    else:
        raise ValueError(f"No valid validation metrics available for epoch {epoch_numbers}.")
    return np.argwhere(np.array(epoch_numbers)==best_chpt['epoch']).item(), best_chpt['val_loss_epoch']

In [26]:
def calculate_average_metrics_mean(base_path = 'logs\CNN-GNN18_mr2k_seeds', start=0, interval=1):
    total_accuracy = []
    total_f1 = []
    total_prec = []
    total_rec = []
    total_loss = []
    
    for i in range(start, start + interval):
        version_path = join(base_path, f'version_{i}')
        checkpoint_path = join(version_path, f'checkpoints')
        onlyfiles  = [f for f in listdir(checkpoint_path) if (isfile(join(checkpoint_path, f)) and 'epoch' in f) ]
        epoch_numbers = [int(re.search(r'\d+', f).group()) for f in onlyfiles]
        best_chpt_id, loss = get_best_chpt(join(version_path, 'metrics.csv'), epoch_numbers)
        print(onlyfiles[best_chpt_id])
        mean_infer_acc, mean_infer_f1, mean_infer_prec, mean_infer_rec, classfier_lightning_model = calculatge_metrics(join(checkpoint_path, f'{onlyfiles[best_chpt_id]}'), test_dataloader)
            
        total_accuracy.append(mean_infer_acc)
        total_f1.append(mean_infer_f1)
        total_prec.append(mean_infer_prec)
        total_rec.append(mean_infer_rec)
        total_loss.append(loss)

    total_accuracy = torch.mean(torch.tensor(total_accuracy))
    total_f1 = torch.mean(torch.tensor(total_f1))
    total_prec = torch.mean(torch.tensor(total_prec))
    total_rec = torch.mean(torch.tensor(total_rec))
    total_loss = torch.mean(torch.tensor(total_loss))
    print(f'total_accuracy: {total_accuracy}')
    print(f'total_f1: {total_f1}')
    print(f'total_prec: {total_prec}')
    print(f'total_rec: {total_rec}')
    print(f'total_loss: {total_loss}')
    return classfier_lightning_model

In [27]:
classfier_lightning_model = calculate_average_metrics_mean(r'logs\CNN-GNN_False_False_False', start=15)

epoch=38-step=1560.pth


c:\Users\fardin\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\utilities\parsing.py:208: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.


total_accuracy: 0.8321874999999999
total_f1: 0.8325216569410598
total_prec: 0.8329931204608162
total_rec: 0.8320517391786908
total_loss: 0.9615753293037416
